In [ ]:
def generate_document_with_guidelines(
    vector_store: VectorStore,
    guidelines: List[str],
    k: int = 3,
    max_new_tokens: int = 512,
    temperature: float = 0.7
) -> str:
    """
    Searches the VectorStore for relevant document chunks,
    then iterates over each guideline to produce a detailed snippet.
    Returns one combined final document with sections for every guideline.
    """
    # Retrieve top-k relevant document chunks from the context
    search_results = vector_store.search("", k=k)  # no user query provided

    # Build a combined context from the top documents
    combined_context_parts = []
    for doc, sim_score in search_results:
        meta_str = ""
        if doc.metadata:
            meta_parts = [f"{key}: {val}" for key, val in doc.metadata.items()]
            meta_str = "\n".join(meta_parts)

        context_str = (
            f"---\nContent:\n{doc.content}\n"
            f"Metadata:\n{meta_str}\n"
            f"Similarity Score: {sim_score:.4f}\n---"
        )
        combined_context_parts.append(context_str)
    combined_context = "\n\n".join(combined_context_parts)

    # Generate a detailed answer for every guideline, including tables if applicable.
    final_snippets = []
    for idx, guideline in enumerate(guidelines, start=1):
        # Notice the prompt now solely relies on context and the guideline.
        prompt = f"""
Context from relevant documents (with metadata):
{combined_context}

Guideline #{idx}: {guideline}

Based on the above context, create a detailed and complete section that fully addresses the guideline. Include all relevant statistics, tables, terminologies, definitions, and any structured data (e.g., tables) if needed.
"""
        snippet = get_completion(
            prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature
        )
        formatted_snippet = f"### GUIDELINE #{idx}: {guideline}\n{snippet.strip()}\n"
        final_snippets.append(formatted_snippet)

    # Combine all guideline-based snippets into a final document
    final_document = "\n\n".join(final_snippets)
    return final_document


# Example usage:
if __name__ == "__main__":
    # Load and index your document from file
    vector_store = load_documents("/commons/copra_share/k152350/ReAct_Testing/metatext.txt")
    
    # List of guidelines
    guidelines = [
        "Briefly describe the business portfolio to which the model applies",
        "Include the products and business segments offered by the business line",
        "Describe the products of the LOB and portfolio to which this model applies.",
        "Describe any current or planned changes in the products, channels, policies, programs, organization, or marketing practices that may impact the model under consideration.",
        "Assess how close the current customer base to the target customer profile is.",
        "Consider whether the customer base is likely to shift over the lifetime of the model.",
        "Specify the current and possible future market conditions and the impact they may have on the portfolio and the model.",
        "Describe the growth of the portfolio over the past 5, 10, X years, both in size and in significance to the balance sheet.",
        "If this is a revalidation or the model replaces an existing model, highlight the key relevant changes on the business between the previous model developments and model validations, and this validation.",
        "Include all the changes related to modeling, such as model framework and theory, variables, data sources, and programs; business changes, such as policy or strategy; environmental changes, such as competitor actions, economic changes, and political or regulatory changes; and any other changes that impact the model, its implementation, evaluation, and usage.",
        "Include a table or summary of the portfolio, product, or business metrics of the business in the most recent and past periods. These can include metrics such as balances, losses, recoveries, number of accounts, average account size, credit limits, etc."
    ]
    
    # Generate the final document answering every guideline in detail
    final_document = generate_document_with_guidelines(vector_store, guidelines)
    print(final_document)
